In [1]:
pip install flask

In [4]:
!pip install -q -U trax

     |████████████████████████████████| 637 kB 5.1 MB/s 
     |████████████████████████████████| 4.9 MB 43.5 MB/s 
     |████████████████████████████████| 462 kB 56.0 MB/s 


In [2]:
pip install ipynb

In [5]:
import numpy as np
from flask import Flask, request, jsonify
import pickle
import trax
from trax import layers as tl
from trax.supervised import training

In [ ]:
import gzip
with gzip.open('/content/drive/MyDrive/model/model.pkl.gz', 'rb') as f:
     model=f.read()

In [13]:
data_dir = 'drive/MyDrive/Data/data'

dialogue_db = {}

vocab_file = "en_32k.subword"
vocab_dir = 'drive/MyDrive/Data/'

In [6]:
def attention(*args, **kwargs):
    
    kwargs['predict_mem_len'] = 120
    
    kwargs['predict_drop_len'] = 120
    
    return tl.SelfAttention(*args, **kwargs)

In [7]:
def ReformerLM(vocab_size = 33000, n_layers = 6, mode = 'train', attention_type = attention):
  model=  trax.models.reformer.ReformerLM(
      vocab_size = vocab_size,
      n_layers = n_layers,
      mode = mode,
      attention_type = attention_type
  )
  return model


In [8]:
bot_model = ReformerLM(mode ='predict')
STARTING_STATE = bot_model.state

In [9]:
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
bot_model.init_from_file('drive/MyDrive/model/chatbot_model1.pkl.gz',
                     weights_only=True, input_signature=shape11)

# save the starting state
STARTING_STATE = bot_model.state

In [16]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [17]:
def generate_dialogue(ReformerLM, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    
    # define the delimiters we used during training
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    
    # initialize detokenized output
    sentence = ''
    
    # token counter
    counter = 0
    
    # output tokens. we insert a ': ' for formatting
    result = [tokenize(': ', vocab_file=vocab_file, vocab_dir=vocab_dir)]
    
    # reset the model state when starting a new dialogue
    ReformerLM.state = model_state
    
    # calls the output generator implemented earlier
    output = ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file=vocab_file, vocab_dir = vocab_dir, temperature=temperature)
    
    # print the starting sentence
    print(start_sentence.split(delimiter_2)[0].strip())
    
    # loop below yields the next tokens until max_len is reached. the if-elif is just for prettifying the output.
    for o in output:
        
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=vocab_file, vocab_dir=vocab_dir)
        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            sentence = ''
            result.clear()
            break 

In [ ]:
!pip install -q -U trax

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
def home():
  bot_model = ReformerLM(mode ='predict')
  STARTING_STATE = bot_model.state
  shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
  bot_model.init_from_file('drive/MyDrive/model/chatbot_model1.pkl.gz',
                     weights_only=True, input_signature=shape11)
  STARTING_STATE = bot_model.state
  return "<h1>GFG is great platform to learn</h1>"
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://76a8-35-197-50-94.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Apr/2022 03:09:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2022 03:09:32] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2022 03:11:32] "GET / HTTP/1.1" 200 -


In [18]:
s = ' Person 1: Are there theatres in town? Person 2: '
t=''
while(t!="exit"):
  s = 'Person 1: '
  t = input()
  s+=t
  s+=' Person 2: '
  STARTING_STATE = bot_model.state
  generate_dialogue(bot_model, model_state=STARTING_STATE, start_sentence=s, vocab_file=vocab_file, vocab_dir=vocab_dir, max_len=120, temperature = 0.2) 


hello
Person 1: hello
Person 2: : How many tickets would you find me one called the gandhi. Person 2: The Kohinoor is a free, would you like to book a table? 


KeyboardInterrupt: ignored